In [11]:
from mrjob.job import MRJob

In [10]:
from mrjob.job import MRJob
from mrjob.step import MRStep

import re

WORD_RE = re.compile(r"[\w']+")


class MRMostUsedWord(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_get_words,
                   combiner=self.combiner_count_words,
                   reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_find_max_word)
        ]



    def mapper_get_words(self, _, line):
        # yield each word in the line
        for word in WORD_RE.findall(line):
            yield (word.lower(), 1) 
            # la clé c'est le mot en lui même et le 1 c'est la valeur
            # le yield c'est comme le return

    # un combiner : c'est avant la phase shuffle de Map Reduce. Il va permettre d'améliorer la performance d'un algorythme
    # si on a 10 fois la même occurence par exemple, on va les combiner pour que ce soit réduit et plus rapide
    # c'est optionnel mais vivement recommandé



    def combiner_count_words(self, word, counts):
        # optimization: sum the words we've seen so far
        yield (word, sum(counts))



# la réduction : un réducer me donne une liste de tuple qui eux vont contenir des clés et des valeurs
# exemple : my_list = [ (2,5), (12,4), (100,3)]
# on fait la même chose que le combiner mais au global. Il va faire le somme du compte de chaque mot

    def reducer_count_words(self, word, counts):
        # send all (num_occurrences, word) pairs to the same reducer.
        # num_occurrences is so we can easily use Python's max() function.
        yield None, (sum(counts), word)
        # None signifie pas de clé



    # discard the key; it is just None
    def reducer_find_max_word(self, _, word_count_pairs):
        # each item of word_count_pairs is (count, word),
        # so yielding one results in key=counts, value=word
        yield max(word_count_pairs)


if __name__ == '__main__':
    MRMostUsedWord.run()

usage: ipykernel_launcher.py [options] [input files]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9013 --control=9011 --hb=9010 --Session.signature_scheme="hmac-sha256" --Session.key=b"56568c32-321c-4b3f-8533-34ca8d6e9e67" --shell=9012 --transport="tcp" --iopub=9014


SystemExit: 2

c:\Users\cathe\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [13]:
my_list = [ (2,5), (12,4), (100,3)]
max_tuple = max(my_list)
max_tuple

(100, 3)